In [1]:
import datetime
import json
import os
import mthree
import numpy as np
from qiskit import IBMQ
from qiskit.providers.ibmq import IBMQBackend
from qiskit.providers.aer import AerSimulator
from qiskit.providers.aer.noise import NoiseModel
from qiskit_experiments.framework import ExperimentEncoder
from qiskit_research.mzm_generation import KitaevHamiltonianExperiment
from qiskit_research.mzm_generation.utils import pick_qubit_layout, orbital_combinations

In [2]:
# get backend

# hardware backend
IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q-internal', group='deployed', project='default')
backend = provider.get_backend('ibm_cairo')

# Aer simulator backend
# noise_model = NoiseModel.from_backend(backend)
# backend = AerSimulator(method='density_matrix', noise_model=noise_model)
# backend = AerSimulator()

In [5]:
# Hamiltonian parameters
n_modes = 5
tunneling_values = [-1.0]
superconducting_values = [1.0]
chemical_potential_values = np.linspace(0.0, 3.0, num=5)
occupied_orbitals_list = list(orbital_combinations(n_modes, threshold=2))

# qubits
qubits, backend_name, error_score = pick_qubit_layout(n_modes, [backend])
# qubits = list(range(n_modes))
print(f'Using qubits {qubits}.')

# number of shots
shots = 100000

# number of shots for readout error calibration
readout_calibration_shots = 100000

# date
date = datetime.datetime.now().isoformat()

# experiment id for saving data
experiment_id = f'{date}_{backend.name()}'

# create experiments
experiment = KitaevHamiltonianExperiment(
    experiment_id=experiment_id,
    backend=backend,
    readout_calibration_date=date,
    qubits=qubits,
    tunneling_values=tunneling_values,
    superconducting_values=superconducting_values,
    chemical_potential_values=chemical_potential_values,
    occupied_orbitals_list=occupied_orbitals_list,
    dynamical_decoupling_sequences=['XY4pm']
)

Using qubits [22, 19, 16, 14, 13].


In [ ]:
%%time

# schedule readout calibration
print('Scheduling readout calibration ...')
mit = mthree.M3Mitigation(backend)
mit.cals_from_system(qubits, shots=readout_calibration_shots, async_cal=True)

# schedule experiment jobs
print(f'Scheduling experiment ...')
# limit number of circuits per job to avoid timeouts
backend.configuration().max_experiments = 150
data = experiment.run(shots=shots)
if isinstance(backend, IBMQBackend):
    # this only works if the Qiskit Experiments database service is available
    data.save()

# save readout calibration
filename = f'data/readout_calibration/{backend.name()}/{date}.json'
os.makedirs(os.path.dirname(filename), exist_ok=True)
print(f'Waiting for readout calibration data ...')
mit.cals_to_file(filename)
print(f'Readout calibration saved.')

Scheduling readout calibration ...
Scheduling experiment ...
You can view the experiment online at https://quantum-computing.ibm.com/experiments/1e8c9b77-2dce-4b1e-a528-9e1ee789a6be
Waiting for readout calibration data ...


In [ ]:
# wait for experiment results
data.block_for_results()

# save data
filename = f'data/{experiment_id}.json'
os.makedirs(os.path.dirname(filename), exist_ok=True)
with open(filename, 'w') as f:
    json.dump(data, f, cls=ExperimentEncoder)

print(experiment_id)